In [26]:
!pip install transformers dataset accelerate torch
from datasets import load_dataset
!pip install -U transformers

ERROR: Could not find a version that satisfies the requirement tranaformers (from versions: none)
ERROR: No matching distribution found for tranaformers


In [9]:
!git clone https://github.com/palaksriv/PRODIGY-GA-01.git

fatal: destination path 'PRODIGY-GA-01' already exists and is not an empty directory.


In [10]:
!ls

PRODIGY-GA-01  sample_data


In [11]:
!ls PRODIGY-GA-01/data

train.txt


In [15]:
dataset=load_dataset('text',data_files={ 'train':"PRODIGY-GA-01/data/train.txt"}) #helps in batching
#rather than having it done manually

In [16]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [18]:
model_name='gpt2'
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 48
    })
})

In [22]:
def tokenize(sample):
  return tokenizer(
      sample['text'],
      truncation=True,
      padding='max_length',
      max_length=128
  )

In [23]:
tokenized_dataset=dataset.map(tokenize,batched=True,remove_columns=['text'])

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [24]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 48
    })
})

###training

In [39]:
from transformers import Trainer, TrainingArguments
training_args=TrainingArguments(
    output_dir='/results',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
)


In [40]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [41]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    data_collator=data_collator
)
trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=48, training_loss=3.0909525553385415, metrics={'train_runtime': 20.656, 'train_samples_per_second': 4.648, 'train_steps_per_second': 2.324, 'total_flos': 6271008768000.0, 'train_loss': 3.0909525553385415, 'epoch': 2.0})

In [44]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


In [45]:
prompt = "<|startoftext|>The night was unusually quiet"
input_ids=tokenizer.encode(prompt,return_tensors='pt')
input_ids = input_ids.to(model.device) #move input_ids to the same device as the model
attention_mask = input_ids.ne(tokenizer.pad_token_id).long()
output=model.generate(input_ids=input_ids,attention_mask=attention_mask,max_length=150,temperature=0.8,top_p=0.95,do_sample=True,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.pad_token_id)
generated_text=tokenizer.decode(output[0],skip_special_tokens=True)
print(generated_text)

<|startoftext|>The night was unusually quiet. She wondered why there hadn't been any people. She knew she had never been to a place where her parents had been able to speak. She had been at the library for months when no one had come to tell her who she really was. The place smelled faintly of her parents' house. It was filled with the books of the young poets, but she had never read. She did not know what she had been doing, but her mother had told her she was sorry for her. "You need a place to stay for a while. You need to be there for a long time. The place you're here is impossible to imagine. And you need to know."




In [47]:
with open('generated_story.txt','w') as f:
  f.write(generated_text)